In [1]:
import pickle
import numpy as np

import glob
import pandas as pd

fl = sorted(glob.glob('../2nd_features/*.pkl'))
for l in fl:
    print("'{}',".format(l))

'../2nd_features/cnn_1_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_2_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_3_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_4_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_5_aug_1_fold5_feat.pkl',
'../2nd_features/incept_aug2_feat.pkl',
'../2nd_features/other_model_gbrt.pkl',
'../2nd_features/other_model_gbrt2.pkl',
'../2nd_features/other_model_gbrt4.pkl',
'../2nd_features/other_model_gbrt5.pkl',
'../2nd_features/other_model_gbrt6.pkl',
'../2nd_features/other_model_gbrt7.pkl',
'../2nd_features/other_model_lgb.pkl',
'../2nd_features/other_model_lgb2.pkl',
'../2nd_features/other_model_lgb4.pkl',
'../2nd_features/other_model_lgb5.pkl',
'../2nd_features/other_model_lgb6.pkl',
'../2nd_features/other_model_lgb7.pkl',
'../2nd_features/other_model_lr.pkl',
'../2nd_features/other_model_lr2.pkl',
'../2nd_features/other_model_lr4.pkl',
'../2nd_features/other_model_lr7.pkl',
'../2nd_features/other_model_rf.pkl',
'../2nd_features/other_model_rf2.pkl',
'../2nd_featur

In [2]:
fl=[
'../2nd_features/cnn_1_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_2_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_3_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_4_aug_1_fold5_feat.pkl',
'../2nd_features/cnn_5_aug_1_fold5_feat.pkl',
'../2nd_features/incept_aug2_feat.pkl',
#'../2nd_features/other_model_gbrt.pkl',
#'../2nd_features/other_model_gbrt2.pkl',
'../2nd_features/other_model_gbrt4.pkl',
#'../2nd_features/other_model_gbrt5.pkl',
'../2nd_features/other_model_gbrt6.pkl',
#'../2nd_features/other_model_gbrt7.pkl',
#'../2nd_features/other_model_lgb.pkl',
#'../2nd_features/other_model_lgb2.pkl',
#'../2nd_features/other_model_lgb4.pkl',
#'../2nd_features/other_model_lgb5.pkl',
#'../2nd_features/other_model_lgb6.pkl',
'../2nd_features/other_model_lgb7.pkl',
#'../2nd_features/other_model_lr.pkl',
#'../2nd_features/other_model_lr2.pkl',
#'../2nd_features/other_model_lr4.pkl',
#'../2nd_features/other_model_lr7.pkl',
#'../2nd_features/other_model_rf.pkl',
#'../2nd_features/other_model_rf2.pkl',
'../2nd_features/other_model_rf4.pkl',
'../2nd_features/other_model_rf5.pkl',
'../2nd_features/other_model_rf6.pkl',
'../2nd_features/other_model_rf7.pkl',
#'../2nd_features/other_model_xgb.pkl',
#'../2nd_features/other_model_xgb2.pkl',
'../2nd_features/other_model_xgb4.pkl',
'../2nd_features/other_model_xgb5.pkl',
#'../2nd_features/other_model_xgb6.pkl',
'../2nd_features/other_model_xgb7.pkl',
'../2nd_features/resnet_aug2_feat.pkl',
'../2nd_features/xception_aug1_feat.pkl',
'../2nd_features/vgg_aug1_new_feat.pkl',

]

def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        #print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 19) (8424, 19) (1604,)


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]

        # train model
        m = LogisticRegression(C=2.0,max_iter=100)
        m.fit(X_train,y_train)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict_proba(X_test))
        curr_tr_loss = log_loss(y_train,m.predict_proba(X_train))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        print('curr train loss',curr_tr_loss,'curr val loss', curr_val_loss)
        
        if test_pred is None:
            test_pred = m.predict_proba(test_x)
            w_test_pred = m.predict_proba(test_x)*curr_r_val_loss
        else:
            curr_pred = m.predict_proba(test_x)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    print(test_pred.shape)
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame(columns=['id','is_iceberg'])
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred[:,1]
        print(submission.head())
        submission.to_csv('../results/all_lr_2nd_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

In [4]:
cv_test(3,42,True)

curr train loss 0.134976036669 curr val loss 0.142475404442
curr train loss 0.135943541319 curr val loss 0.136805816454
curr train loss 0.132212590501 curr val loss 0.149582244435
local average valid loss 0.142954488444 val loss std 0.00522694423792
(8424, 2)
         id  is_iceberg
0  5941774d    0.026166
1  4023181e    0.815790
2  b20200e4    0.169140
3  e7f018bb    0.988920
4  4371c8c3    0.129657


In [5]:
cv_test(5,42,True)

curr train loss 0.131395457817 curr val loss 0.154123939613
curr train loss 0.139104405726 curr val loss 0.125006950472
curr train loss 0.134640173657 curr val loss 0.142171076217
curr train loss 0.137233101737 curr val loss 0.133268794747
curr train loss 0.13069436414 curr val loss 0.153769178417
local average valid loss 0.141667987893 val loss std 0.0114015723184
(8424, 2)
         id  is_iceberg
0  5941774d    0.025169
1  4023181e    0.811583
2  b20200e4    0.172494
3  e7f018bb    0.989335
4  4371c8c3    0.126693


In [6]:
cv_test(3,252,True)

curr train loss 0.130580277332 curr val loss 0.150758766839
curr train loss 0.131001051954 curr val loss 0.149683963465
curr train loss 0.14163206477 curr val loss 0.1272540072
local average valid loss 0.142565579168 val loss std 0.0108358041592
(8424, 2)
         id  is_iceberg
0  5941774d    0.025665
1  4023181e    0.818931
2  b20200e4    0.179054
3  e7f018bb    0.988816
4  4371c8c3    0.137122


In [7]:
cv_test(5,66,True)

curr train loss 0.13580227204 curr val loss 0.139042530668
curr train loss 0.129285251587 curr val loss 0.162957132207
curr train loss 0.132813565881 curr val loss 0.150405165349
curr train loss 0.133612631774 curr val loss 0.142832364316
curr train loss 0.140817535622 curr val loss 0.116686608256
local average valid loss 0.142384760159 val loss std 0.0152270499426
(8424, 2)
         id  is_iceberg
0  5941774d    0.025273
1  4023181e    0.812443
2  b20200e4    0.175201
3  e7f018bb    0.989348
4  4371c8c3    0.130657


In [8]:
cv_test(10,42,True)

curr train loss 0.131792322555 curr val loss 0.168389402275
curr train loss 0.134645912135 curr val loss 0.140857066894
curr train loss 0.136227760566 curr val loss 0.130252509946
curr train loss 0.13716845305 curr val loss 0.117393073724
curr train loss 0.130967151903 curr val loss 0.171977337881
curr train loss 0.138404105708 curr val loss 0.111224358117
curr train loss 0.131975035873 curr val loss 0.164276842844
curr train loss 0.139548231621 curr val loss 0.103139240221
curr train loss 0.132924442568 curr val loss 0.155599822681
curr train loss 0.132638435714 curr val loss 0.156376407386
local average valid loss 0.141948606197 val loss std 0.0238432764997
(8424, 2)
         id  is_iceberg
0  5941774d    0.024921
1  4023181e    0.809696
2  b20200e4    0.169715
3  e7f018bb    0.989642
4  4371c8c3    0.125599
